In [1]:
import sys
 
sys.path.insert(0, "../")

from EnsembleFramework import Framework

In [2]:
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops

dataset_name = 'Cora'
split = "public"
dataset = Planetoid(root='/tmp/Cora', name=dataset_name, split=split)
dataset.transform = T.NormalizeFeatures()

features =  dataset[0].x
y =  dataset[0].y

test =  dataset[0].test_mask
train = dataset[0].train_mask
val =  dataset[0].val_mask

edge_index = dataset[0].edge_index 
edge_index = add_self_loops(edge_index)[0]

In [3]:
hops_to_best_configs = {0: {'inter_layer_normalize': True,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 1: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 2: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.00011111111111111112},
 3: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.00022222222222222223},
 4: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 5: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 6: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0003333333333333334},
 7: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 8: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00011111111111111112},
 9: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00022222222222222223},
 10: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00011111111111111112},
 15: {'inter_layer_normalize': True,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 20: {'inter_layer_normalize': True,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 30: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00044444444444444447},
 40: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.0003333333333333334},
 50: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.0}}

In [4]:
from torch.nn.functional import normalize
def user_function(kwargs):
    return  normalize(kwargs["original_features"] + kwargs["summed_neighbors"], p=2.0, dim = 1) #normalize(kwargs["original_features"] + kwargs["summed_neighbors"], p=2.0, dim = 1)
hops_list = [0,1,2,3,4,5,6,7,8,9,10,15,20,30,40,50]
framework = Framework([user_function for i in hops_list], 
                     hops_list=hops_list, ## to obtain best for local neighborhood
                     clfs=[],
                     gpu_idx=0,
                     handle_nan=0.0,
                    attention_configs=[hops_to_best_configs[hop] for hop in hops_list])
new_features_list = framework.get_features(features, edge_index, None)

/home/dwalke/git/graph_aware_ml/svc_classifier/../EnsembleFramework.py:146: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:230.)
  summed_exp_score = torch.zeros_like(exp_score).scatter(0, target,exp_score, reduce="add")


In [6]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import ParameterGrid
from tqdm.notebook import tqdm
from hyperopt import fmin, tpe, hp,STATUS_OK

kernel = ["linear", "poly", "rbf", "sigmoid"]
degree = [i for i in range(2, 5)]
gamma = ["scale", "auto"]

lr_choices = {
    'kernel': kernel,
    'degree': degree,
    'gamma': gamma,
}

space = {
    **{key: hp.choice(key, value) for key, value in lr_choices.items()},
    'tol': hp.loguniform('tol', -11, -3),
    'C': hp.uniform('C', 0.0, 150)
}

clf = SVC
hops_to_params=dict({})
hops_to_score=dict({})

for i, hops in tqdm(enumerate(hops_list)):
    new_features = new_features_list[i].cpu()
    
    def objective(params):
        score = 0
        try:
            model = clf(**params, random_state = 42, probability= True, cache_size=800)
            model.fit(new_features[train], y[train])
            y_pred = model.predict(new_features[val])
            score = accuracy_score(y[val], y_pred)
        except Exception:
            print(Exception)
            print(params)
            score = -100
        return {'loss': -score, 'status': STATUS_OK}
    
    best_params = fmin(objective, space, algo=tpe.suggest, max_evals=1_00, verbose=1)

    for key,value in lr_choices.items():
        best_params[key] = value[best_params[key]]

    best_model = clf(**best_params, random_state = 42, probability= True, cache_size=800)
    best_model.fit(new_features[train], y[train])
    y_pred = best_model.predict(new_features[val])
    
    hops_to_params[hops] = best_params
    hops_to_score[hops] = accuracy_score(y[val], y_pred)

0it [00:00, ?it/s]


100%|███████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.36trial/s, best loss: -0.604]

100%|███████████████████████████████████████████████████████████| 100/100 [00:14<00:00,  7.12trial/s, best loss: -0.692]

100%|███████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.67trial/s, best loss: -0.778]

100%|███████████████████████████████████████████████████████████| 100/100 [00:14<00:00,  6.78trial/s, best loss: -0.796]

100%|███████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.71trial/s, best loss: -0.798]

100%|███████████████████████████████████████████████████████████| 100/100 [00:19<00:00,  5.20trial/s, best loss: -0.802]

100%|████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  8.15trial/s, best loss: -0.81]

100%|███████████████████████████████████████████████████████████| 100/100 [00:14<00:00,  6.84trial/s, best loss: -0.814]

100%|██████████████████

In [7]:
import pandas as pd
df = pd.DataFrame(columns=["best_params"], index=hops_to_params.keys())
df["best_params"] = hops_to_params.values()
df.to_csv("normalized_origin_features_plus_summed_neighbors.csv")
df

,best_params
0,"{'C': 17.5365956122971, 'degree': 2, 'gamma': ..."
1,"{'C': 99.50472947355532, 'degree': 3, 'gamma':..."
2,"{'C': 45.40310436626846, 'degree': 2, 'gamma':..."
3,"{'C': 56.56980378351197, 'degree': 4, 'gamma':..."
4,"{'C': 125.56300909127322, 'degree': 4, 'gamma'..."
5,"{'C': 104.8196894499217, 'degree': 2, 'gamma':..."
6,"{'C': 2.104612736327283, 'degree': 2, 'gamma':..."
7,"{'C': 2.715982239920301, 'degree': 2, 'gamma':..."
8,"{'C': 68.6461625884557, 'degree': 4, 'gamma': ..."
9,"{'C': 3.587848395210898, 'degree': 2, 'gamma':..."


In [8]:
hops_to_score

{0: 0.604,
 1: 0.692,
 2: 0.778,
 3: 0.796,
 4: 0.798,
 5: 0.802,
 6: 0.81,
 7: 0.814,
 8: 0.802,
 9: 0.812,
 10: 0.814,
 15: 0.794,
 20: 0.796,
 30: 0.812,
 40: 0.81,
 50: 0.812}